# Scraping - Detik
Scraping was done on 5th September 2021 9:02 AM

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests
from tqdm.notebook import tqdm_notebook
import pandas as pd
import os
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
def readpage(url):
    req = requests.get(url, verify=False)
    page = bs(req.content, 'lxml')
    return page

In [3]:
driver = webdriver.Edge('msedgedriver.exe')

In [4]:
website = 'https://www.detik.com/search/searchall?query=covid&siteid=2&sortby=time&page=1'
driver.get(website)

In [5]:
%%time
judul_detik = []
date_detik = []
link_detik = []

# Page Open
for i in tqdm_notebook(range(150)):
    page = bs(driver.page_source, 'lxml')
    
    # Define Group
    group = page.find('div', {'class':'list media_rows list-berita'})
    articles = group.findChildren('article', recursive=False)
        
    # Judul
    judul = [article.find('h2', {'class':'title'}).get_text().strip() for article in articles]
    judul_detik.extend(judul)
    
    # Date
    for article in articles:
        date = article.find('span', {'class':'date'}).text
        date_detik.append(date)
    
    # Links
    links = [article.find('a')['href'] for article in articles]
    link_detik.extend(links)
    
    # Click Next Page
    try:
        next_page = driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/a[7]')
        next_page.click()
    except:
        next_page = driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/a[6]')
        next_page.click()
        


Wall time: 2min 40s


In [6]:
# Make into DF
dtk_df = pd.DataFrame(list(zip(judul_detik, date_detik, link_detik)))
dtk_df.columns = ['judul', 'tanggal', 'link']

In [7]:
dtk_df

,judul,tanggal,link
0,Malaysia Heran Corona di RI Turun Drastis Lebi...,"detikHealthMinggu, 05 Sep 2021 09:01 WIB",https://health.detik.com/berita-detikhealth/d-...
1,"Pesepeda Ramaikan Kawasan Sudirman Pagi Ini, P...","detikNewsMinggu, 05 Sep 2021 08:39 WIB",https://news.detik.com/berita/d-5710393/pesepe...
2,Upaya Memvaksinasi Warga di Pegunungan Papua,"detikNewsMinggu, 05 Sep 2021 08:35 WIB",https://news.detik.com/detiktv/d-5710390/upaya...
3,"Dear Nikita Mirzani, Dipo Latief Pesan Jangan ...","detikHotMinggu, 05 Sep 2021 08:25 WIB",https://hot.detik.com/celeb/d-5710378/dear-nik...
4,Erick Thohir Buka-bukaan Rencana Produksi Vaks...,"detikFinanceMinggu, 05 Sep 2021 08:00 WIB",https://finance.detik.com/berita-ekonomi-bisni...
...,...,...,...
1345,UNICEF: Banyak Siswa Harus Berbagi Smartphone ...,"detikEduSenin, 30 Agu 2021 13:30 WIB",https://www.detik.com/edu/sekolah/d-5701927/un...
1346,Honor Bupati Jember dalam Pemakaman COVID-19 S...,"detikNewsSenin, 30 Agu 2021 13:29 WIB",https://news.detik.com/berita-jawa-timur/d-570...
1347,"Jalani Vaksinasi COVID-19, Sejumlah Ibu Hamil ...","detikNewsSenin, 30 Agu 2021 13:25 WIB",https://news.detik.com/berita-jawa-barat/d-570...
1348,Satgas COVID Dikritik Tak Tegas Tangani Kerumu...,"detikNewsSenin, 30 Agu 2021 13:22 WIB",https://news.detik.com/berita/d-5701951/satgas...


In [8]:
# Put into CSV
dtk_df.to_csv('detik_data.csv', index=False)